In [1]:
import os
from pathlib import Path

metadata_raw = open(f"{Path.home()}/Downloads/train.json", "r").readlines()

In [2]:
import json


metadata_list = list(json.loads("".join(metadata_raw)))

In [6]:
import numpy as np


os.makedirs("shards", exist_ok=True)

with open("./shards/shard_all.json", "w") as f1:
  for i in range(0, len(metadata_list), 5000):
    shard_id = i // 5000
    with open(f"./shards/shard_{shard_id}.json", "w") as f:
      for j in range(i, min(i + 5000, len(metadata_list))):
        image = f"shard_{shard_id}/{metadata_list[j]['image']}"
        conversations_vi = metadata_list[j]["vi"]
        if len(conversations_vi) > 0:
          if np.random.rand() < 0.5:
            conversations_vi[0]["value"] = f"<image>\n{conversations_vi[0]['value']}"
          else:
            conversations_vi[0]["value"] = f"{conversations_vi[0]['value']}\n<image>"

        data = {
          "id": metadata_list[j]["id"],
          "image": image,
          "conversations": conversations_vi
        }
        f.write(json.dumps(data, ensure_ascii=False) + "\n")
        f1.write(json.dumps(data, ensure_ascii=False) + "\n")

In [7]:
from vividbot.data.processor.huggingface import HuggingFaceProcessor


hf_processor = HuggingFaceProcessor()

hf_processor.zip_and_upload_dir(
  dir_path="./shards",
      repo_id="Vividbot/llava-instruct-150k-vi",
      path_in_repo="shards.zip",
      repo_type="dataset",
      overwrite=True,
    
)

shards.zip: 100%|██████████| 121M/121M [00:15<00:00, 8.05MB/s] 


In [3]:
import numpy as np


os.makedirs("shards", exist_ok=True)

BASE_DATA_PATH = f"{Path.home()}/llava-instruct-150k-vi"

with open(f"{BASE_DATA_PATH}/metadata.json", "w") as f:
  for shard in os.listdir(f"{BASE_DATA_PATH}/shards"):
    with open(f"{BASE_DATA_PATH}/shards/{shard}", "r") as f1:
      for line in f1:
        f.write(line)